In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import cv2
from PIL import Image
import tensorflow as tf
import random
import keras
import keras.backend as K
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
train_file = '../Training'
valid_file = '../Validation'
test_file  = '../Testing'

train_datagen = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2)
valid_datagen = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2)
test_datagen  = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2)


train_generator = train_datagen.flow_from_directory(train_file,
                                                 target_size=(500, 500),
                                                 batch_size=32,
                                                 class_mode='categorical')

valid_generator = train_datagen.flow_from_directory(valid_file,
                                                 target_size=(500, 500),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_file,
                                            target_size=(500, 500),
                                            batch_size=32,
                                            class_mode='categorical')

Found 1132 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Found 260 images belonging to 2 classes.
here


In [14]:

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=1, activation='relu', input_shape = (500,500,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='softmax'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))


model.summary()


def recall(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
     recall = true_positives / (possible_positives + K.epsilon())
     return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

sgd = keras.optimizers.SGD(lr=0.000001, momentum=0.9, decay=0.0001, nesterov=True)
model.compile(optimizer= sgd, loss='binary_crossentropy', metrics=['accuracy', recall, precision])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 500, 500, 32)      128       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 250, 250, 32)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 249, 249, 32)      4128      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 124, 124, 32)      0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 122, 122, 64)      18496     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 61, 61, 64)        0         
__________

"#check how to feed the input train set\ninputs = Input(shape=(224,))\nlayer1 = Dense(64, activation='relu')(inputs)\nlayer2 = Dense(64, activation='relu')(layer1)\npredictions = Dense(10, activation='softmax')(layer2)\nmodel = Model(inputs=inputs, outputs=predictions)\n\ndef custom_loss(y_true,y_pred):\n    \n    alpha = 0.23\n    # Create a loss function that adds a weight to the predictions\n    return K.mean(K.square((1 - alpha)y_pred - (alpha)y_true))\n    \n# Compile the model\nmodel.compile(optimizer='adam',\n              loss=custom_loss(y_true,y_pred), # Call the loss function with the selected layer\n              metrics=['accuracy'])\n\n"

In [15]:
epochs = 30
 
mpd = model.fit_generator(train_generator,
                          steps_per_epoch = None,                                    
                          epochs = epochs,                                         
                          validation_data = valid_generator,
                          validation_steps = None) 

Epoch 1/3
36/36 [==============================] - 1291s 36s/step - loss: 0.6944 - acc: 0.3906 - recall: 0.3906 - precision: 0.3906 - val_loss: 0.6938 - val_acc: 0.1290 - val_recall: 0.1290 - val_precision: 0.1290
Epoch 2/3
36/36 [==============================] - 998s 28s/step - loss: 0.6943 - acc: 0.3897 - recall: 0.3897 - precision: 0.3897 - val_loss: 0.6937 - val_acc: 0.1290 - val_recall: 0.1290 - val_precision: 0.1290
Epoch 3/3
36/36 [==============================] - 996s 28s/step - loss: 0.6941 - acc: 0.4210 - recall: 0.4210 - precision: 0.4210 - val_loss: 0.6936 - val_acc: 0.1290 - val_recall: 0.1290 - val_precision: 0.1290


In [13]:
valid_generator.batch_size = 1
score = model.evaluate_generator(valid_generator)

test_generator.reset()

scores = model.predict_generator(test_generator, len(test_generator))
print("Loss: ", score[0], "Accuracy: ", score[1])

predicted_class_indices=np.argmax(scores,axis=1)
print(predicted_class_indices)

label = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labelss[k] for k in predicted_class_indices]

filenames = test_generator.filenames
results = pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

print(results)


/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `evaluate_generator` call to the Keras 2 API: `evaluate_generator(<keras_pre..., 10, use_multiprocessing=False)`
  # Remove the CWD from sys.path while we load stuff.


Loss:  0.6932463347911835 Accuracy:  0.4
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
                                   Filename    Predictions
0    Non_Resistant/PIL-160_3dayLBCR-1.1.jpg  Non_Resistant
1    Non_Resistant/PIL-160_3dayLBCR-1.2.jpg  Non_Resistant
2    Non_Resistant/PIL-160_3dayLBCR-1.3.jpg  Non_Resistant
3    Non_Resistant/PIL-160_3dayLBCR-1.4.jpg  Non_Resistant
4    Non_Resistant/PIL-160_3dayLBCR-2.1.jpg  Non_Resistant
5    Non_Resistant/PIL-160_3dayLBCR-2.2.jpg  Non_Resistant
6    Non_Resistan

"\n\n\nprint('here')\n\npredictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_files]\nprint(np.array(predictions))\nprint(np.array(test_targets))\n\ncompare_predictions = np.equal(np.array(predictions),np.array(test_targets))\ntest_accuracy = 100*np.sum(compare_predictions)/len(predictions)\nprint('Test accuracy: %.4f%%' % test_accuracy)\n\n# Plot training & validation accuracy values\nplt.plot(history.history['acc'])\n#plt.plot(history.history['val_acc'])\nplt.title('Model accuracy')\nplt.ylabel('Accuracy')\nplt.xlabel('Epoch')\nplt.legend(['Train', 'Test'], loc='upper left')\nplt.show()\n\n# Plot training & validation loss values\nplt.plot(history.history['loss'])\n#plt.plot(history.history['val_loss'])\nplt.title('Model loss')\nplt.ylabel('Loss')\nplt.xlabel('Epoch')\nplt.legend(['Train', 'Test'], loc='upper left')\nplt.show()\n\nprint(history.history.values)\nprint(history.history['acc'])\n\n\n"